In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import os
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename

In [135]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET]

train_paths = glob('../feature/raw_main/*_train.gz')
test_paths = glob('../feature/raw_main/*_test.gz')

train_paths = [path for path in train_paths 
               if path.count('DT') 
#                or path.count('day_no') 
               or path.count('Amt')
               or path.count('Fraud') 
               or path.count('D')
               or path.count('C')
               or path.count('V')
#                or path.count('card')
#                or path.count('addr')
#                or path.count('Reg')
#                or path.count('P_email')
#                or path.count('R_email')
#                or path.count('M')
#                or path.count('Product')
              ]
test_paths = [path for path in test_paths 
               if path.count('DT') 
#                or path.count('day_no') 
               or path.count('Amt')
               or path.count('Fraud') 
               or path.count('D')
               or path.count('C')
               or path.count('V')
#                or path.count('card')
#                or path.count('addr')
#                or path.count('Reg')
#                or path.count('P_email')
#                or path.count('R_email')
#                or path.count('M')
#                or path.count('Product')
              ]

train_df = parallel_load_data(train_paths)
test_df = parallel_load_data(test_paths)

In [31]:
train_df.head()

,D15,V7,V210,V225,V203,V3,V61,V252,TransactionID,V170,...,V304,V272,D13,V267,V303,TransactionAmt,V91,D8,V2,ProductCD
0,0.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,2987000,NaN,...,0.0,NaN,NaN,NaN,0.0,68.5,0.0,NaN,1.0,W
1,0.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2987001,NaN,...,0.0,NaN,NaN,NaN,0.0,29.0,0.0,NaN,NaN,W
2,315.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,2987002,NaN,...,0.0,NaN,NaN,NaN,0.0,59.0,0.0,NaN,1.0,W
3,111.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2987003,NaN,...,0.0,NaN,NaN,NaN,0.0,50.0,0.0,NaN,NaN,W
4,NaN,NaN,0.0,0.0,0.0,NaN,NaN,1.0,2987004,1.0,...,1.0,0.0,NaN,0.0,1.0,50.0,NaN,NaN,NaN,H


In [32]:
train_df.set_index(COLUMN_ID, inplace=True)
list_path = glob('../output/0830_ieee__same_user__pattern-user_keys__card*')
for path in list_path:
    fname = path[56:].replace('csv', '').replace(r'.', '')
    same_user = pd.read_csv(path).set_index(COLUMN_ID)
    train_df[f'same_user__{fname}']  = same_user['same_user_id']

In [38]:
startdate = datetime.datetime(2017,11,4)
train_df['datetime'] = train_df['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x)))
train_df['month'] = train_df['datetime'].map(lambda x: x.month)
train_df['date'] = train_df['datetime'].map(lambda x: x.date())

In [45]:
cols_V = sorted([int(col[1:]) for col in train_df.columns if col.startswith('V')])
cols_V = ['V' + str(col) for col in cols_V]
cols_D = sorted([int(col[1:]) for col in train_df.columns if col.startswith('D')])
cols_D = ['D' + str(col) for col in cols_D]
cols_C = sorted([int(col[1:]) for col in train_df.columns if col.startswith('C')])
cols_C = ['C' + str(col) for col in cols_C]
cols_same = sorted([col for col in train_df.columns if col.startswith('same')])
cols_trans = sorted([col for col in train_df.columns if col.startswith('Trans')])
cols_sort = cols_same + ['datetime', COLUMN_TARGET, 'ProductCD'] + cols_trans + cols_D + cols_C + cols_V

同じと思われるユーザーで、C系の差分を見る。どれだけ増加するものなのか？変わらないものなのか？

In [73]:
pd.set_option('max_rows', 300)

c_min = train_df.groupby('same_user__addr_pemail_M')[cols_C].min()
c_min.columns = [f"min_{col}" for col in c_min.columns]
c_max = train_df.groupby('same_user__addr_pemail_M')[cols_C].max()
c_max.columns = [f"max_{col}" for col in c_max.columns]
c_diff = c_max.join(c_min)

for col in cols_C:
    c_diff[f'diff_{col}'] = c_diff[f'max_{col}'] - c_diff[f"min_{col}"]

cols_C_diff = [col for col in c_diff.columns if col.count('diff_')]
c_diff.reset_index(inplace=True)
c_diff_max = c_diff.groupby('same_user__addr_pemail_M')[cols_C_diff].max()

tmp = pd.DataFrame(index=np.arange(200))
multi_c_diff_max = c_diff_max.loc[multi_user_idx]
for col in c_diff_max:
    tmp = tmp.join(multi_c_diff_max[col].value_counts(), how='left')
tmp

,diff_C1,diff_C2,diff_C4,diff_C5,diff_C6,diff_C7,diff_C8,diff_C9,diff_C10,diff_C11,diff_C12,diff_C13,diff_C14
0,7132.0,5893.0,8900.0,8179.0,7298.0,8977.0,8540.0,7893.0,8544.0,7437.0,8350.0,1555.0,7084.0
1,1117.0,1270.0,237.0,474.0,1093.0,201.0,355.0,726.0,385.0,942.0,742.0,1568.0,1325.0
2,274.0,506.0,61.0,145.0,241.0,39.0,115.0,162.0,167.0,239.0,107.0,1284.0,241.0
3,121.0,413.0,38.0,58.0,113.0,30.0,103.0,89.0,73.0,109.0,26.0,1790.0,93.0
4,63.0,253.0,10.0,33.0,46.0,8.0,56.0,38.0,38.0,61.0,14.0,982.0,51.0
5,47.0,178.0,10.0,30.0,36.0,7.0,32.0,23.0,13.0,29.0,4.0,528.0,36.0
6,31.0,96.0,4.0,17.0,32.0,3.0,20.0,26.0,17.0,22.0,5.0,302.0,27.0
7,21.0,71.0,5.0,10.0,27.0,4.0,16.0,16.0,14.0,23.0,4.0,227.0,18.0
8,19.0,52.0,6.0,15.0,25.0,6.0,11.0,18.0,7.0,18.0,6.0,131.0,22.0
9,16.0,33.0,8.0,13.0,28.0,8.0,7.0,19.0,5.0,27.0,3.0,93.0,21.0
